## Task description:


In [2]:
requirements = """Requirements:
I need a single-page application with a header, followed by a section with a digital clock and a footer:
- The header should contain the text: "Hello from the agent-metaverse side! Here is the result of my first experiment:"
- The clock section should be centrally aligned and contain a digital clock that updates every second. The background color of the div should be black and the text color should be white.
- The footer should contain the text: "Powered by synapse labs" in font 14px, italic, and color black.
"""

## Validator: Project manager head
Responsible for:
- generating a task plan
- breaking the task plan into structured actions of (description, acceptance criteria)

In [4]:
import os
import openai
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']

project_plan_template = """
You are a professional project manager; the goal is to break down the list of requirements in triple backticks into tasks to be given to a software engineer.
Your tasks should contain:
- task description: The work that the programmer will need to perform
- acceptance criteria: What is the acceptance criteria to define the success of the task to be performed
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", project_plan_template),
    ("user", "{input}")
])

model = ChatOpenAI(model_name="gpt-4", temperature=0)

project_plan_generation_chain = prompt | model

project_plan = project_plan_generation_chain.invoke({"input": requirements})
project_plan

AIMessage(content='Task 1: Create a Single-Page Application\n- Task Description: Set up a new single-page application using your preferred framework (React, Angular, Vue, etc.). The application should have a basic structure with a header, a main section, and a footer.\n- Acceptance Criteria: The application runs successfully without errors and displays a blank page with a header, main section, and footer.\n\nTask 2: Add Text to Header\n- Task Description: Add the text "Hello from the agent-metaverse side! Here is the result of my first experiment:" to the header of the application.\n- Acceptance Criteria: The specified text is visible in the header of the application.\n\nTask 3: Create a Digital Clock\n- Task Description: In the main section of the application, create a digital clock that updates every second. The clock should be centrally aligned.\n- Acceptance Criteria: A digital clock is visible in the main section of the application, updates every second, and is centrally aligned.\

In [5]:
print(project_plan.content)

Task 1: Create a Single-Page Application
- Task Description: Set up a new single-page application using your preferred framework (React, Angular, Vue, etc.). The application should have a basic structure with a header, a main section, and a footer.
- Acceptance Criteria: The application runs successfully without errors and displays a blank page with a header, main section, and footer.

Task 2: Add Text to Header
- Task Description: Add the text "Hello from the agent-metaverse side! Here is the result of my first experiment:" to the header of the application.
- Acceptance Criteria: The specified text is visible in the header of the application.

Task 3: Create a Digital Clock
- Task Description: In the main section of the application, create a digital clock that updates every second. The clock should be centrally aligned.
- Acceptance Criteria: A digital clock is visible in the main section of the application, updates every second, and is centrally aligned.

Task 4: Style the Clock Sect

In [6]:
from pydantic import BaseModel, Field
from typing import List

class Task(BaseModel):
    """A task to be performed by a software engineer, containing a description and acceptance criteria"""
    description: str = Field(description="The descrption of the task to be performed")
    acceptance_criteria: str = Field(description="The acceptance criteria to define the success of the task to be performed")

class ProjectPlan(BaseModel):
    """The project plan of a software engineer sprint, defined by the raw plain text of the initial rationale and the list of tasks to be performed"""
    plan: str = "A full copy of the original prompt used to generate the task plan"
    tasks: List[Task] = Field(description="The list of tasks to be performed")

In [7]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

task_extraction_functions = [convert_pydantic_to_openai_function(ProjectPlan)]
task_extraction_model = model.bind(functions=task_extraction_functions, function_call={"name" : "ProjectPlan"})

task_organization_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the project plan alogside the tasks structured in task description and acceptance criteria, do not guess or invent information."),
    ("human", "{project_plan}")
])

task_organization_chain = task_organization_prompt | task_extraction_model | JsonOutputFunctionsParser(keyname="ProjectPlan")

raw_project_plan = task_organization_chain.invoke({"project_plan": project_plan})
raw_project_plan['plan'] = project_plan.content

mapped_project_plan = ProjectPlan(**raw_project_plan)
mapped_project_plan

ProjectPlan(plan='Task 1: Create a Single-Page Application\n- Task Description: Set up a new single-page application using your preferred framework (React, Angular, Vue, etc.). The application should have a basic structure with a header, a main section, and a footer.\n- Acceptance Criteria: The application runs successfully without errors and displays a blank page with a header, main section, and footer.\n\nTask 2: Add Text to Header\n- Task Description: Add the text "Hello from the agent-metaverse side! Here is the result of my first experiment:" to the header of the application.\n- Acceptance Criteria: The specified text is visible in the header of the application.\n\nTask 3: Create a Digital Clock\n- Task Description: In the main section of the application, create a digital clock that updates every second. The clock should be centrally aligned.\n- Acceptance Criteria: A digital clock is visible in the main section of the application, updates every second, and is centrally aligned.\n

In [8]:
print(mapped_project_plan)

plan='Task 1: Create a Single-Page Application\n- Task Description: Set up a new single-page application using your preferred framework (React, Angular, Vue, etc.). The application should have a basic structure with a header, a main section, and a footer.\n- Acceptance Criteria: The application runs successfully without errors and displays a blank page with a header, main section, and footer.\n\nTask 2: Add Text to Header\n- Task Description: Add the text "Hello from the agent-metaverse side! Here is the result of my first experiment:" to the header of the application.\n- Acceptance Criteria: The specified text is visible in the header of the application.\n\nTask 3: Create a Digital Clock\n- Task Description: In the main section of the application, create a digital clock that updates every second. The clock should be centrally aligned.\n- Acceptance Criteria: A digital clock is visible in the main section of the application, updates every second, and is centrally aligned.\n\nTask 4: St

In [9]:
write_code_template = """
You are a professional engineer with deep knowledge on web development; the main goal is to write code that is clean, elegant, modular, easy to read and maintain
ATTENTION: Use '##' to SPLIT SECTIONS, not '#'. Output format carefully referenced "Format example".

## Code:  Write code with triple quoto, based on the following list and context.
1. Do your best to implement THIS ONLY ONE FILE. ONLY USE EXISTING API. IF NO API, IMPLEMENT IT.
2. Requirement: Based on the task, implement one following code file, note to return only in code form, your code will be part of the entire project, so please implement complete, reliable, reusable code snippets
3. Attention1: If there is any setting, ALWAYS SET A DEFAULT VALUE, ALWAYS USE STRONG TYPE AND EXPLICIT VARIABLE.
4. Think before writing: What should be implemented and provided in to fill the given acceptance criteria?

-----
# Task
{task}
-----
## Format example
-----
## Code: file.html:
```html
## html code here...
...
```
-----
"""

coding_prompt = ChatPromptTemplate.from_messages([
    ("system", write_code_template),
    ("human", "{task}")
])

coding_chain = coding_prompt | model

coding_output = coding_chain.invoke({"task": mapped_project_plan.tasks[0].description})
coding_output

AIMessage(content='## Code: App.js (React):\n```javascript\nimport React from \'react\';\n\nconst App = () => {\n  return (\n    <div>\n      <Header />\n      <Main />\n      <Footer />\n    </div>\n  );\n};\n\nconst Header = () => {\n  return (\n    <header>\n      <h1>Welcome to our Single Page Application</h1>\n    </header>\n  );\n};\n\nconst Main = () => {\n  return (\n    <main>\n      <p>This is the main section of our application.</p>\n    </main>\n  );\n};\n\nconst Footer = () => {\n  return (\n    <footer>\n      <p>© 2022 Our Company. All rights reserved.</p>\n    </footer>\n  );\n};\n\nexport default App;\n```\n\n## Code: index.js (React):\n```javascript\nimport React from \'react\';\nimport ReactDOM from \'react-dom\';\nimport App from \'./App\';\n\nReactDOM.render(\n  <React.StrictMode>\n    <App />\n  </React.StrictMode>,\n  document.getElementById(\'root\')\n);\n```\n\n## Code: index.html (React):\n```html\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset

In [10]:
coding_output.content

'## Code: App.js (React):\n```javascript\nimport React from \'react\';\n\nconst App = () => {\n  return (\n    <div>\n      <Header />\n      <Main />\n      <Footer />\n    </div>\n  );\n};\n\nconst Header = () => {\n  return (\n    <header>\n      <h1>Welcome to our Single Page Application</h1>\n    </header>\n  );\n};\n\nconst Main = () => {\n  return (\n    <main>\n      <p>This is the main section of our application.</p>\n    </main>\n  );\n};\n\nconst Footer = () => {\n  return (\n    <footer>\n      <p>© 2022 Our Company. All rights reserved.</p>\n    </footer>\n  );\n};\n\nexport default App;\n```\n\n## Code: index.js (React):\n```javascript\nimport React from \'react\';\nimport ReactDOM from \'react-dom\';\nimport App from \'./App\';\n\nReactDOM.render(\n  <React.StrictMode>\n    <App />\n  </React.StrictMode>,\n  document.getElementById(\'root\')\n);\n```\n\n## Code: index.html (React):\n```html\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8" />\n    <

In [11]:
import re
import logging as logger
import ast

class CodeParser:
    @classmethod
    def parse_block(cls, block: str, text: str) -> str:
        blocks = cls.parse_blocks(text)
        for k, v in blocks.items():
            if block in k:
                return v
        return ""

    @classmethod
    def parse_blocks(cls, text: str):
        # 首先根据"##"将文本分割成不同的block
        blocks = text.split("##")

        # 创建一个字典，用于存储每个block的标题和内容
        block_dict = {}

        # 遍历所有的block
        for block in blocks:
            # 如果block不为空，则继续处理
            if block.strip() != "":
                # 将block的标题和内容分开，并分别去掉前后的空白字符
                block_title, block_content = block.split("\n", 1)
                block_dict[block_title.strip()] = block_content.strip()

        return block_dict

    @classmethod
    def parse_code(cls, block: str, text: str, lang: str = "") -> str:
        if block:
            text = cls.parse_block(block, text)
        pattern = rf"```{lang}.*?\s+(.*?)```"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            code = match.group(1)
        else:
            logger.error(f"{pattern} not match following text:")
            logger.error(text)
            # raise Exception
            return text  # just assume original text is code
        return code

    @classmethod
    def parse_str(cls, block: str, text: str, lang: str = ""):
        code = cls.parse_code(block, text, lang)
        code = code.split("=")[-1]
        code = code.strip().strip("'").strip('"')
        return code

    @classmethod
    def parse_file_list(cls, block: str, text: str, lang: str = "") -> list[str]:
        # Regular expression pattern to find the tasks list.
        code = cls.parse_code(block, text, lang)
        # print(code)
        pattern = r"\s*(.*=.*)?(\[.*\])"

        # Extract tasks list string using regex.
        match = re.search(pattern, code, re.DOTALL)
        if match:
            tasks_list_str = match.group(2)

            # Convert string representation of list to a Python list using ast.literal_eval.
            tasks = ast.literal_eval(tasks_list_str)
        else:
            raise Exception
        return tasks

In [12]:
code = CodeParser.parse_code(block="", text=coding_output.content)
print(code)

import React from 'react';

const App = () => {
  return (
    <div>
      <Header />
      <Main />
      <Footer />
    </div>
  );
};

const Header = () => {
  return (
    <header>
      <h1>Welcome to our Single Page Application</h1>
    </header>
  );
};

const Main = () => {
  return (
    <main>
      <p>This is the main section of our application.</p>
    </main>
  );
};

const Footer = () => {
  return (
    <footer>
      <p>© 2022 Our Company. All rights reserved.</p>
    </footer>
  );
};

export default App;



In [13]:
# Ugly way to extract the file name out of the content, to be improved

#print(coding_output.content)
# Regular expression to match the file name
regex = r"## Code: (\S+):"

# Find the match
match = re.search(regex, coding_output.content)
filename = match.group(1) if match else None
filename

In [18]:
review_code_template = """
You are a professional software engineer, and your main task is to review the code. You need to ensure that the code is elegantly designed and modularized, easy to read and maintain.
ATTENTION: Use '##' to SPLIT SECTIONS, not '#'. Output format carefully referenced "Format example".

## Code Review: Based on the following context and code, and following the check list, Provide key, clear, concise, and specific code modification suggestions, up to 5.
```
1. Check 0: Is the code implemented as per the requirements?
2. Check 1: Are there any issues with the code logic?
3. Check 2: Does the existing code follow the "Data structures and interface definitions"?
4. Check 3: Is there a function in the code that is omitted or not fully implemented that needs to be implemented?
5. Check 4: Does the code have unnecessary or lack dependencies?
6. Check 5: Does the code matches the acceptance criteria?
```

## Rewrite Code: {filename} Base on "Code Review" and the source code, rewrite code with triple quotes. Do your utmost to optimize THIS SINGLE FILE. 
-----
# Context
Task: {task}
Acceptance criteria: {acceptance_criteria}

## Code: {filename}
```
{code}
```
-----

## Format example
-----
## Code Review
1. The code ...
2. ...
3. ...
4. ...
5. ...

## Rewrite Code: filename
```js
// code here
...
```
"""

code_review_prompt = ChatPromptTemplate.from_messages([
    ("system", review_code_template),
    ("human", "{task}, {acceptance_criteria}, {filename}, {code}")
])


code_review_chain = code_review_prompt | model

code_review_output = code_review_chain.invoke({"task": mapped_project_plan.tasks[0].description, "acceptance_criteria": mapped_project_plan.tasks[0].acceptance_criteria, "filename": filename, "code": code})
code_review_output

AIMessage(content='## Code Review\n1. The code is implemented as per the requirements. It sets up a single-page application with a header, main section, and footer.\n2. There are no issues with the code logic. The components are correctly defined and used.\n3. The existing code follows the "Data structures and interface definitions". It uses functional components which is a common pattern in React.\n4. There is no function in the code that is omitted or not fully implemented. All components are fully implemented.\n5. The code does not have unnecessary dependencies. It only imports \'react\', which is necessary for the code to run.\n6. The code matches the acceptance criteria. It runs successfully without errors and displays a blank page with a header, main section, and footer.\n\n## Rewrite Code: App.js\n```js\nimport React from \'react\';\n\nconst Header = () => (\n  <header>\n    <h1>Welcome to our Single Page Application</h1>\n  </header>\n);\n\nconst Main = () => (\n  <main>\n    <

In [19]:
code_review_output = CodeParser.parse_code(block="", text=code_review_output.content)
print(code_review_output)

import React from 'react';

const Header = () => (
  <header>
    <h1>Welcome to our Single Page Application</h1>
  </header>
);

const Main = () => (
  <main>
    <p>This is the main section of our application.</p>
  </main>
);

const Footer = () => (
  <footer>
    <p>© 2022 Our Company. All rights reserved.</p>
  </footer>
);

const App = () => (
  <div>
    <Header />
    <Main />
    <Footer />
  </div>
);

export default App;



In [20]:
import tqdm

memory = None
coding_output_history_log = []
coding_review_history_log = []

for task in tqdm.tqdm(mapped_project_plan.tasks):
    if memory is None:
        coding_output = coding_chain.invoke({"task": task.description})
    else:
        coding_output = coding_chain.invoke({"task": task.description + f'\nBase code to be evolved: {memory}'})
    code = CodeParser.parse_code(block="", text=coding_output.content)
        
    # Regular expression to match the file name
    regex = r"## Code: (\S+):"

    # Find the match
    match = re.search(regex, coding_output.content)
    filename = match.group(1) if match else None

    code_review_output = code_review_chain.invoke({"task": task.description, "acceptance_criteria": task.acceptance_criteria, "filename": filename, "code": code})
    code_review_output = CodeParser.parse_code(block="", text=code_review_output.content)
    

    coding_output_history_log.append(coding_output)
    coding_review_history_log.append(code_review_output)
    memory = code_review_output

 20%|██        | 1/5 [00:28<01:54, 28.59s/it]